# Setup


In [1]:
import ee
import urllib.request
from shapely.geometry import shape, Point
import pandas as pd
import json
import math

In [2]:
#ee.Authenticate()
ee.Initialize()

# Get geojson border data

In [ ]:
target_country = ""
coordinates_target_country = None

# read json file with counties
with open("countries.geojson", encoding='utf-8') as f:
    countries_geojson = json.load(f)

for feature in countries_geojson['features']:
    if feature['properties']['ADMIN']==target_country:
        coordinates_target_country = feature["geometry"]
        print(coordinates_target_country)


In [ ]:
# Variety of Configuations
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 1024 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB
imageDimensions = '1024x1024' #Set thumbnail image size (can't be too big, or you run into problems)
nir_bands = ['B8'] #Includes the bands for nir
swirB11_bands = ['B11']
swirB12_bands = ['B12']
tif_size = 1024
image_creation_time_start = '2015-07-01'
image_creation_time_end = '2016-07-01'
r_earth = 6371000
# 10x10 meter per pixel
sentinel_resolution = 10

# construct polygon from coordinates to shaply object
polygon = shape(coordinates_target_country)
# get bounds of polygon
minx, miny, maxx, maxy = polygon.bounds

# start point bottom left:
x,y = minx,miny
id=0
# iterate over all columns:
while y < maxy:
  while x < maxx:
    west,south = x,y
    east = x + (tif_size*sentinel_resolution / r_earth) * (180 / math.pi) / math.cos(y * math.pi/180)
    north = y  + (tif_size*sentinel_resolution / r_earth) * (180 / math.pi)
    # if one of the edges of the box is in the polygon, export the image
    if polygon.contains(Point(west, south)) or polygon.contains(Point(east, north)):
      print(f"{x} - {y}")
      # request geotif
      box = ee.Geometry.BBox(west,south,east,north)
      # Import Sentinel dataset
      # either COPERNICUS/S2_SR or COPERNICUS/S2
      s2 = (ee.ImageCollection("COPERNICUS/S2_SR")
        .filterBounds(box)
        .sort('CLOUDY_PIXEL_PERCENTAGE')
        .filterDate(image_creation_time_start, image_creation_time_end)
        .filter(ee.Filter.contains('.geo', box))
        .first()
      )
      
      s2Vis = {
        'region': box,
        'crs': (s2.select('B4').projection()),
        'dimensions': imageDimensions,
        'format': 'jpg',
        'bands': vis_bands,
        'gamma': 1.4,
        'min': 0,
        'max': 10000
      }
      try:
        s2_url = (s2.getThumbURL(s2Vis))
        print(s2_url)
        #Change the location where the images are saved by replacing "content" with the location in your Google Drive
        s2_name = "{}/preview/S2_{}_{}_{}.jpg".format("country_export", id,x, y)
        s2_tif_name = "S2_{}_{}_{}".format(id,x, y)
        id = id + 1
        urllib.request.urlretrieve(s2_url, s2_name)
        print(s2_tif_name)
        task = ee.batch.Export.image.toDrive(
          image = s2.select('B2','B3','B4','B8','B11','B12'),
          description=s2_tif_name,
          folder='country_export',
          region=box,
          dimensions=imageDimensions,
          crs=(s2.select('B4').projection())
        )
        task.start()
      except:
        print("No data found! Skipping...")
      
    # increase x (long)
    x = x + (tif_size*sentinel_resolution / r_earth) * (180 / math.pi) / math.cos(y * math.pi/180)
  # reset x
  x = minx
  # increase y (lat)
  y  = y  + (tif_size*sentinel_resolution / r_earth) * (180 / math.pi)
